In [1]:
import pandas as pd
import numpy as np

In [2]:
response = pd.read_csv('./data/response.csv')
response=response.drop(['id'],axis = 1)
response= response.drop_duplicates()

hotel = pd.read_csv('./data/hotel.csv')

req = pd.read_csv('./data/request.csv')
req = req.drop(['created_at'],axis = 1)
req = req.drop_duplicates()

avatar = pd.read_csv('./data/avatar.csv')
sample = pd.read_csv('./data/sample.csv')

In [4]:
# Request in a city
reqt = req[['avatar_id','language','city','date']]
reqt = reqt.drop_duplicates()
reqt['city_num'] = reqt.groupby(['avatar_id','city']).cumcount()+1
req = req.join(reqt.set_index(['avatar_id','language','city','date']), on=['avatar_id','language','city','date'],how='left', rsuffix='_right')
req = req.drop(['city'],axis = 1)
req.head()

,id,avatar_id,language,date,mobile,city_num
0,1,1,french,2,True,1
1,2,2,italian,40,False,1
2,3,2,italian,40,False,1
3,4,2,italian,30,False,1
4,5,2,italian,30,False,1


In [4]:
def dummy(data,col,prefix=''):
    df = data
    dd = pd.get_dummies(data[col],prefix=prefix)
    df = pd.concat([df,dd],axis = 1)
    df = df.drop([col],axis =1)
    return df

In [5]:
hotel = dummy(hotel,'brand',prefix='b_')
hotel = dummy(hotel,'city',prefix='c_')
hotel = dummy(hotel,'group',prefix='g_')

In [6]:
req = dummy(req,'language',prefix='l_')

In [7]:
response = response.join(req.set_index('id'), on='request_id', validate='m:1')
response = response.join(hotel.set_index('id'), on='hotel_id', validate='m:1')
response[['mobile','parking','pool']]=response[['mobile','parking','pool']].astype(int)
response = response.sort_values(by='request_id')

In [8]:
response = response[['request_id','avatar_id', 'date','mobile','city_num','hotel_id', 'price', 'stock', 
        'parking', 'pool', 'children_policy','l__austrian', 'l__belgian', 'l__bulgarian',
       'l__croatian', 'l__cypriot', 'l__czech', 'l__danish', 'l__dutch',
       'l__estonian', 'l__finnish', 'l__french', 'l__german', 'l__greek',
       'l__hungarian', 'l__irish', 'l__italian', 'l__latvian', 'l__lithuanian',
       'l__luxembourgish', 'l__maltese', 'l__polish', 'l__portuguese',
       'l__romanian', 'l__slovakian', 'l__slovene', 'l__spanish', 'l__swedish','b__8 Premium', 'b__Ardisson',
       'b__Boss Western', 'b__Chill Garden Inn', 'b__Corlton', 'b__CourtYord',
       'b__Ibas', 'b__Independant', 'b__J.Halliday Inn', 'b__Marcure',
       'b__Morriot', 'b__Navatel', 'b__Quadrupletree', 'b__Royal Lotus',
       'b__Safitel', 'b__Tripletree', 'c__amsterdam', 'c__copenhagen',
       'c__madrid', 'c__paris', 'c__rome', 'c__sofia', 'c__valletta',
       'c__vienna', 'c__vilnius', 'g__Accar Hotels', 'g__Boss Western',
       'g__Chillton Worldwide', 'g__Independant', 'g__Morriott International',
       'g__Yin Yang']]

In [9]:
response = response.drop(['request_id','avatar_id'],axis=1)

In [11]:
index = sample['id']
sample = sample.drop(['id','scrape_request_id'],axis=1)
sample['request_id']=sample['order_requests']
sample=sample.drop(['order_requests'],axis=1)
sample.head()

,avatar_id,language,city,date,mobile,stock,hotel_id,request_id
0,1,romanian,vilnius,21,False,46,161,1
1,1,romanian,vilnius,21,False,32,187,1
2,1,romanian,vilnius,21,False,12,279,1
3,1,romanian,vilnius,21,False,10,395,1
4,1,romanian,vilnius,21,False,42,488,1


In [12]:
samt = sample[['avatar_id','language','city','date']]
samt = samt.drop_duplicates()
samt['city_num'] = samt.groupby(['avatar_id','city']).cumcount()+1
sample = sample.join(samt.set_index(['avatar_id','language','city','date']), on=['avatar_id','language','city','date'],how='left', rsuffix='_right')
sample.head()

,avatar_id,language,city,date,mobile,stock,hotel_id,request_id,city_num
0,1,romanian,vilnius,21,False,46,161,1,1
1,1,romanian,vilnius,21,False,32,187,1,1
2,1,romanian,vilnius,21,False,12,279,1,1
3,1,romanian,vilnius,21,False,10,395,1,1
4,1,romanian,vilnius,21,False,42,488,1,1


In [13]:
sample = sample.drop(['city'],axis=1)
sample = dummy(sample,'language',prefix='l_')
sample = sample.join(hotel.set_index('id'), on='hotel_id', validate='m:1')
sample = sample.drop(['avatar_id', 'request_id'],axis=1)
sample = sample.sort_index()

In [14]:
X = response.drop(['price'],axis=1)
X_1 = X[X['city_num']==1]
X_1 = X_1.drop(['city_num'],axis=1)

y = response['price']
y_1 = y[X_1.index]

sample_1 = sample[sample['city_num']==1]
sample_1 = sample_1.drop(['city_num'],axis=1)


In [16]:
from sklearn.linear_model import SGDRegressor

cat = SGDRegressor()
cat.fit(X,y)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGDRegressor()

In [38]:
!mkdir -p './submission/1911'
pred_cat = cat.predict(sample)
pred_cat= pd.DataFrame(pred_cat)
sub_cat = pd.concat([index,pred_cat],axis=1)
sub_cat['index']=sub_cat['id']
sub_cat=sub_cat.drop(['id'],axis=1)
sub_cat['price']=sub_cat[0]
sub_cat=sub_cat.drop(0,axis=1)
sub_cat.to_csv('./submission/1911/submission_cat_1300.csv',index=False)

In [39]:
!mkdir -p './model/811'
cat.save_model('./model/811/cat.cbm')
cat_1.save_model('./model/811/cat_1.cbm')

In [40]:
pred_cat= cat.predict(sample)
pred_cat= pd.DataFrame(pred_cat)
sub_cat = pd.concat([index,pred_cat],axis=1)
sub_cat['index']=sub_cat['id']
sub_cat=sub_cat.drop(['id'],axis=1)
sub_cat['price']=sub_cat[0]
sub_cat=sub_cat.drop(0,axis=1)
sub_cat.to_csv('./submission/811/submission_cat_1608_seul.csv',index=False)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
